In [1]:
from collections import Counter, defaultdict, deque, namedtuple
from copy import deepcopy
import functools
import inspect
import json
import os
from pathlib import Path
import pickle
from pprint import pp, pprint
import re
import sys
import time
from typing import Dict, List

import numpy as np
import pandas as pd
import plotly.express as px

from colorutils import Color

from dotenv import load_dotenv
from tqdm import tqdm

from aic_nlp_utils.json import read_jsonl, read_json, write_json, write_jsonl, process_to_jsonl
from aic_nlp_utils.pycfg import parse_pycfg_args, read_pycfg
%load_ext autoreload
%autoreload 2

from prompt_opt.utils import *

sys.path.append("/home/drchajan/devel/python/FC/automated-fact-checking")

os.environ['VLLM_WORKER_MULTIPROC_METHOD']='spawn'
load_dotenv()

/home/drchajan/devel/python/FC/VENV/vllm/lib/python3.11/site-packages/aic_nlp_utils/json.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


True

# AlignScore

In [3]:
def sample_random(src_path, k, rng, lang="en", query_fmt="html", tag_document="document1", tag_claim="document2"):
    assert lang in ["en", "cs"]
    assert query_fmt in ["html", "json"]
    src_path = Path(src_path)
    raw_data = []
    for fname in src_path.iterdir():
        if fname.is_file():
            fdata = read_jsonl(fname)
            raw_data += list(rng.choice(fdata, k, replace=False))
    
    data = []
    for e in raw_data:
        if query_fmt == "html":
            if lang == "en":
                query = f"<{tag_document}>{e['document']}</{tag_document}>\n<{tag_claim}>{e['claim']}</{tag_claim}>"
            else:
                query = f"<{tag_document}>{e['translated_ces_document']}</{tag_document}>\n<{tag_claim}>{e['translated_ces_claim']}</{tag_claim}>"
        else:
            if lang == "en":
                query = {tag_document: e['document'], tag_claim: e['claim']}
            else:
                query = {tag_document: e['translated_ces_document'], tag_claim: e['translated_ces_claim']}
            
        answer = {"class": e["label"]}
        data.append({"query": query, "answer": answer}) 
    
    rng.permutation(data)
    pos_idxs = [i for i, e in enumerate(data) if e["answer"]["class"] == 1]
    neg_idxs = [i for i, e in enumerate(data) if e["answer"]["class"] == 0]
    
    # interleave positive and negative samples so every even-sized prefix is balanced
    interleaved_data = []
    for p, n in zip(pos_idxs, neg_idxs):
        interleaved_data.append(data[p])
        interleaved_data.append(data[n])
    
    labels = [e["answer"]["class"] for e in interleaved_data]
    
    print(f"# positive samples: {np.sum(labels)}/{len(interleaved_data)}")
    return interleaved_data

data = sample_random("/mnt/data/factcheck/AlignScore-data/Benchmarks/deepl_summac/val", k=100, rng=np.random.RandomState(1234), query_fmt="json")
write_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/alignscore_bench_val_json_query_masked_tags.jsonl", data)

# data = sample_random("/mnt/data/factcheck/AlignScore-data/Benchmarks/deepl_summac/val", k=100, rng=np.random.RandomState(1234), query_fmt="json", tag_document="document", tag_claim="claim")
# write_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/alignscore_bench_val_json_query_named_tags.jsonl", data)

# positive samples: 236/472


# Libgen

In [14]:
from libgen_api import LibgenSearch

s = LibgenSearch()
# results = s.search_author("Floyd")
results = s.search_title("music modes")
pp(results)
# results = s.search_title("Pride and Prejudice")
# item_to_download = results[0]
# download_links = s.resolve_download_links(item_to_download)
# pp(download_links)

[{'ID': '733677',
  'Author': 'Gioseffo Zarlino',
  'Title': 'On the Modes: Part Four of Le Istitutioni Harmoniche, 1558 (Music '
           'Theory Translation Series) (Pt. 4)',
  'Publisher': 'Yale Univ Pr',
  'Year': '1983',
  'Pages': '150',
  'Language': 'English',
  'Size': '3 Mb',
  'Extension': 'pdf',
  'Mirror_1': 'http://library.lol/main/2C5079BF31ADF249E9ADC34C1021307C',
  'Mirror_2': 'http://libgen.li/ads.php?md5=2C5079BF31ADF249E9ADC34C1021307C',
  'Mirror_3': 'https://library.bz/main/edit/2C5079BF31ADF249E9ADC34C1021307C'},
 {'ID': '2654496',
  'Author': 'Brent,Jeff',
  'Title': 'Modalogy: Scales, Modes & Chords: The Primordial Building Blocks '
           'of Music',
  'Publisher': 'Hal Leonard',
  'Year': '2011;2013',
  'Pages': '',
  'Language': 'English',
  'Size': '2 Mb',
  'Extension': 'epub',
  'Mirror_1': 'http://library.lol/main/FE37ECAA1E27A865393C62FB6FC2D662',
  'Mirror_2': 'http://libgen.li/ads.php?md5=FE37ECAA1E27A865393C62FB6FC2D662',
  'Mirror_3': 'https:/

# People

In [ ]:
data = read_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/people_roles_raw.jsonl")
len(data)

20

In [13]:
pf(data[2]["answer"])

{'Markéta Vaňková': ['kandidátka na brněnskou primátorku', 'členka ODS', 'advokátka vymáhající exekuce pro ostravský
dopravní podnik'], 'Andrej Babiš': ['premiér ČR', 'předseda ANO'], 'Petr Fiala': ['předseda ODS']}


This type of variable key objects (patternProperties in JSON schema) is not yet supported by ObjectAligner. Let's transform to list of dicts. Note names of key will give hints...

In [14]:
def transform_to_list(data):
    for e in data:
        e["answer"] = [{"name": k, "roles": v} for k, v in e["answer"].items()]

transform_to_list(data)

In [16]:
data[2]

{'query': 'Nezastupuji žádné kmotry, ODS se posunula, říká kandidátka na brněnskou primátorku Vaňková\n\nPřekvapivý obrat ve vývoji povolebních jednání v Brně označil premiér Andrej Babiš (ANO) za megapodraz. Jeho hnutí ANO totiž podle všeho zůstane v opozici, přestože volby vyhrálo – a primátorkou se pravděpodobně stane Markéta Vaňková z ODS.\n\n„ODS se vymezuje jako alternativa k hnutí ANO, zejména na celostátní úrovni... Na nižších úrovních je třeba vždy zvažovat konkrétní situaci, nicméně Brno je velké město a cítím od našich voličů, že preferují koalici bez ANO,“ vysvětluje Vaňková.\n\nNěkteří komentátoři ale připravovanou koalici ODS, KDU-ČSL, České pirátské strany (Piráti) a ČSSD nevítají s nadšením. „Nezastupuji žádné kmotry. Strana se posunula, což je naprosto zřejmé a občané to vidí. Líbí se mi, kam ODS pod vedením Petra Fialy směřuje,“ reaguje.\n\nPozornost vzbudil i fakt, že Vaňková v minulosti jako advokátka vymáhala exekuce pro ostravský dopravní podnik. „Zásadní je, že n

In [17]:
write_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/people_roles.jsonl", data)

# Events

In [2]:
data = read_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/events.jsonl")

In [8]:
len(data)

20

In [7]:
pf(data[0]["answer"])

[{'event': 'V únoru 2022 EU navrhuje přijetí více sankcí vůči ruským představitelům, kteří se podíleli na uznání
nezávislosti separatistických regionů na východě Ukrajiny.', 'people': [2374, 1784]}, {'event': 'Maďarsko zatím sankce
nepodpořilo.', 'people': []}, {'event': 'Viktor Orbán je vnímán jako spojenec ruského prezidenta Vladimira Putina.',
'people': [6301, 1600]}]


# SiR 1.0

In [4]:
def import_brat_annotation(sir_path, doc_id):
    txt = Path(sir_path, f"{doc_id}.txt").read_text()
    ann = Path(sir_path, f"{doc_id}.ann").read_text()
    
    entities = {}
    relations = {}
    
    pattern_entity = re.compile(r'(\S+)\s+(\S+)\s+(\d+)\s+(\d+)\s+(.+)')
    for line in ann.splitlines():
        if line.startswith("R"):
            id_, type_, arg1, arg2 = line.split()
            assert type_ == "attribution", type_
            arg1 = arg1.split(":")[1]
            arg2 = arg2.split(":")[1]
            relations[id_] = {"arg1": arg1, "arg2": arg2}
        else:
            match = pattern_entity.match(line)
            if match:
                id_, type_, sidx, eidx, span = match.groups()
                sidx = int(sidx)
                eidx = int(eidx)
                assert span == txt[sidx:eidx], (span, txt[sidx:eidx])
                entities[id_] = {"type": type_, "start": sidx, "end": eidx, "span": span}
            else:
                assert False, line
                
    # order by start index and rename the entities
    ents = list(entities.items())
    ents.sort(key=lambda e: e[1]["start"])
    id2newid = {e[0]: f"T{i+1}" for i, e in enumerate(ents)}
    entities = [{"id": id2newid[e[0]], **e[1]} for e in ents]
    
    # fix entity ids in relations
    for r in relations.values():
        r["arg1"] = id2newid[r["arg1"]]
        r["arg2"] = id2newid[r["arg2"]]
        
    # now sort relations by their first argument
    rels = list(relations.items())
    rels.sort(key=lambda e: int(e[1]["arg1"][1:]))
    relations = [{"id": f"R{i+1}", **e[1]} for i, e in enumerate(rels)]
    
    return {"query": txt, "answer": {"entities": entities, "relations": relations}}

# import_brat_annotation("/mnt/data/factcheck/ufal/SiR1.0/data/triple_manual", "doc-5457987.xml")
import_brat_annotation("/mnt/data/factcheck/ufal/SiR1.0/data/triple_manual", "doc-5753090.xml")

{'query': 'Veslaři si pochvalují domácí atmosféru na mistrovství Evropy v Račicích \nSnad jen na domácím šampionátu zažijí veslaři něco podobného. \nNa startu jim organizátoři přejí mnoho štěstí v češtině a v zádech mají velkou podporu domácího publika. \nTu snad využijí v sobotním programu skifař Ondřej Synek a dvojka bez kormidelníka pro postup do finále. \nMyslím si, že podpora bude veliká. \nCo jsem slyšel, tak lístků už se prodalo dost. \nSnad to bude dobrý," řekl Radiožurnálu Jakub Podrazil z dvojky bez kormidelníka, \nPrávě česká dvojka bez kormidelníka bude dnes podporu domácích fanoušků potřebovat. \nV sobotu dopoledne pojede v opravě o postup do finále. \nNa mě to tady dýchá z každé strany. \nProtože když jedu na start, tak tam všichni mluví česky. \nBójkář, co mi drží loď, na mě mluví česky a přeje mi štěstí. \nTohle se stává jenom na mistrovství republiky, ne na takových závodech," komentuje výhodu domácí atmosféry nejlepší český skifař Ondřej Synek \nBojkář mi říkal, že mi

In [3]:
def select_and_rearragne_entity_types(data, must_contain, rng):
    new_data = []
    for e in data:
        entities = e["answer"]["entities"]
        types = [ent["type"] for ent in entities]
        types = set(types)
        
        select = 0
        for c in must_contain:
            if c in types:
                select += 1
        
        for t in types:
            if t not in must_contain and t != "PHRASE":
                select = 0
                
        if "PHRASE" not in types:
            select = 0
            
        if select > 0:
            types = list(types)
            types.sort()
            # print(i, select, types)
            new_data.append((select, e))
            rng.shuffle(new_data)
            new_data.sort(key=lambda e: e[0], reverse=True)
    new_data = [e[1] for e in new_data]
    return new_data


def import_sir(sir_path, rng, relations=True, spans=True, ids=True):
    data = []
    for fname in Path(sir_path).glob('*.ann'):
        doc_id = fname.name[:-4]
        ex = import_brat_annotation(sir_path, doc_id)
        if not spans:
            ents = ex["answer"]["entities"]
            for e in ents:
                del e["start"]
                del e["end"]
        if not ids:
            ents = ex["answer"]["entities"]
            for e in ents:
                del e["id"]
        if not relations:
            del ex["answer"]["relations"]
        data.append(ex)
        
    # this rearanges the samples so samples with most hits in must_contain_samples are first
    # sort-of stratification
    # for triple_manual the first roughly 20 samples cover nicely 'official-non-political', 'official-political', 'anonymous-partial'
    # there must be at least one PHRASE in all samples
    data = select_and_rearragne_entity_types(data,
                                             must_contain=['official-non-political', 'official-political', 'anonymous-partial'],
                                             rng=rng)
    
    return data
    

# subset = "single"
# subset = "double_unified"
subset = "triple_manual"
data = import_sir(f"/mnt/data/factcheck/ufal/SiR1.0/data/{subset}", rng=np.random.RandomState(1234))
write_jsonl(f"/home/drchajan/devel/python/FC/long_sum/data/labeled_datasets/sir1.0_{subset}.jsonl", data)

data = import_sir(f"/mnt/data/factcheck/ufal/SiR1.0/data/{subset}", rng=np.random.RandomState(1234), relations=False)
write_jsonl(f"/home/drchajan/devel/python/FC/long_sum/data/labeled_datasets/sir1.0_{subset}_no_relations.jsonl", data)

data = import_sir(f"/mnt/data/factcheck/ufal/SiR1.0/data/{subset}", rng=np.random.RandomState(1234), relations=False, spans=False, ids=False)
write_jsonl(f"/home/drchajan/devel/python/FC/long_sum/data/labeled_datasets/sir1.0_{subset}_no_relations_no_spans_no_ids.jsonl", data)



In [4]:
data = read_jsonl(f"/home/drchajan/devel/python/FC/long_sum/data/labeled_datasets/sir1.0_{subset}_no_relations_no_spans_no_ids.jsonl")

In [17]:
def import_sir_phrases(sir_path, rng):
    data = []
    for fname in Path(sir_path).glob('*.ann'):
        doc_id = fname.name[:-4]
        ex = import_brat_annotation(sir_path, doc_id)
        entities = ex["answer"]["entities"]
        phrases = set()
        for e in entities:
            if e["type"] == "PHRASE":
                phrases.add(e["span"])
        phrases = sorted(list(phrases))
        data.append({"query": ex["query"], "answer": phrases})
    data = rng.permutation(data)
    return data
        
subset = "triple_manual"
data = import_sir_phrases(f"/mnt/data/factcheck/ufal/SiR1.0/data/{subset}", np.random.RandomState(1234))

write_jsonl(f"data/labeled_datasets/sir1.0_{subset}_phrases.jsonl", data)


In [10]:
len(data)

46

In [20]:
for i, e in enumerate(data):
    print(i, e["answer"])

0 ['podle', 'popisuje', 'upozorňuje', 'zdůrazňuje', 'zmiňuje', 'říká']
1 ['dodala', 'připomíná', 'reagují', 'upozorňují', 'uvedla', 'řekla']
2 ['podle', 'varovali', 'zveřejnily']
3 ['konstatovali', 'prohlásila', 'se domnívá']
4 ['dodala', 'kritizoval', 'mluvil', 'nabídli', 'řekl', 'říkal']
5 ['Vyplývá']
6 ['Uvedla', 'podle']
7 ['informova', 'napsal', 'podle', 'prohlásil', 'připustil', 'řekl']
8 ['Informovala', 'Podle', 'napsal', 'se ví']
9 ['Myslím si', 'jsem slyšel', 'komentuje', 'nepřeceňuje', 'reprodukuje', 'řekl']
10 ['Dodala', 'Oznámila', 'informovala', 'oznámilo', 'píše', 'uvedla']
11 ['Podle']
12 ['Podle', 'Prý', 'zpochybnil', 'řekl']
13 ['Podle', 'říká']
14 ['popisuje', 'prohlásil', 'upozorňuje', 'zdůrazňuje', 'říká']
15 ['podle', 'prohlásil']
16 []
17 ['nazývá', 'upozorňuje']
18 ['informoval', 'uvedl', 'řekl']
19 ['podle', 'řekl']
20 ['informova', 'uvedl']
21 ['Podle', 'dodal', 'informoval', 'řekl', 'žertoval']
22 []
23 ['doporučuje', 'nabádá', 'píše', 'stojí']
24 ['Podle', 'p

In [6]:
schema = {
  "type": "object",
  "properties": {
    "entities": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "id": { "type": "string"},
          "type": { "type": "string"},
          "start": { "type": "integer"},
          "end": { "type": "integer"},
          "span": { "type": "string"}
        },
        "required": ["id", "type", "start", "end", "span"]
      }
    },
    "relations": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "id": { "type": "string"},
          "arg1": { "type": "string" },
          "arg2": { "type": "string" }
        },
        "required": ["id", "arg1", "arg2"]
      }
    }
  },
  "required": ["entities", "relations"],
  "additionalProperties": False
}

In [18]:
def check_entity_types(data):
    cnt = Counter()
    for i, e in enumerate(data):
        entities = e["answer"]["entities"]
        types = [ent["type"] for ent in entities]
        types = list(set(types))
        types.sort()
        print(i, types)
        cnt.update(types)
    pf(cnt)
    
check_entity_types(data[:])

TypeError: list indices must be integers or slices, not str

In [41]:
data[3]

{'query': 'Kulturní tipy v době koronaviru: Cirk la Putyka v ulicích či online koncert Iva Kahánka \nČesko přijalo kvůli koronaviru řadu přísných opatření. \nKvůli nim jsou mimo jiné zavřená divadla, kina, koncertní či výstavní sály nebo muzea. \nUmělci se ale často přesunuli na internet. \nKulturní redakce Radiožurnálu a iROZHLAS.cz proto nabízí několik tipů, co je ve čtvrtek možné online sledovat nebo poslouchat. \nKdyž nemůžou lidé za umělci, musí umělci za lidmi. \nTo je teď nové heslo souboru Cirk la Putyka, který zahajuje sérii venkovních uměleckých aktivit. \nTímto mottem cirkusové company se ale už od vyhlášení nouzového stavu řídí v podstatě celá kulturní scéna. \nI ve čtvrtek můžou fanoušci navštívit online koncerty nebo si z obývacího pokoje udělat biograf. \nI když Cirk la Putyka několikrát vystoupil pomocí livestreamu, rozhodl se tentokrát opustit online prostor a přesunout se do pražských ulic. \nJejich pódiem jsou tak nově parkoviště nebo dvorky a hledištěm balkóny nemoc